In [1]:
import data
import evaluation
dataset = data.Data("config.json", "nagini_examples/lseg_finetune")
ev = evaluation.Evaluation(dataset, "codellama", predictor_endpoint="huggingface-pytorch-inference-2024-03-17-14-31-56-589")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/omkar/.config/sagemaker/config.yaml


### Evaluating the finetuned mar6 model

Training data was:
- list: [prepend, append, join_lists, contains, drop, reverse, merge]
- tree: [val_head, count, sum, contains, inorder, min, mirror, subtree]
- lseg: [lemma_append, lemma_assoc, prepend, remove_first, contains, contains_iter, insert, append, index_of]

Held out:
- list: [insert_sorted ❌, drop_iter ❌]
- tree: [insert ❌, height ✅]
- lseg: [remove_last ❌, insert_iter ❌, reverse ❌]

Additionally evaluated on:
- list: [remove_first ✅, remove_last ✅, split ✅, index_of ✅, insert ✅, count ✅, insertion_sort ❌, merge_sort ❌]
- tree: [min_depth ✅]
- lseg: [insert_sorted ❌, insertion_sort ❌, count ✅, split ❌, merge ❌, merge_sort ❌]

Datasets to use: list_mar13_finetune, lseg_finetune, tree_finetune.

In [ ]:
ev.run_eval(k=5, n=3, key="lseg")

### Failure modes:

list::drop_iter: uses undefined lemma_extend => translation error => does not learn with error depth. Possibly because no "translation error" in the training set? try higher temperature?

list::insert_sorted: Requires(is_list(node) and Acc(node.val) and Acc(node.next)) Uncallable!

list::insertion_sort: try renaming insert_sorted to insert_node_in_sorted_list(node, head) and variables inside insertion_sort appropriately.

list::merge_sort: extra fold unfolds. try giving specs of dependent methods in the prompt => does not help.

-----------------

tree::insert: if key < node.key: without Unfold/Unfolding => does not learn with error depth. Try higher temperature.

~~tree::subtree: 1. extra Unfold after return statement 2. when 1. solved, extra Fold (does not understand that permissions are to be leaked). Solved at temperature 1.5 at k=2.~~

-----------
lseg::remove_last
```python
    if Unfolding(lseg(first, last), first.next is last):
        return first
```
misses a Fold(first, first) required to satisfy postcondition `Ensures(lseg(first, Result()))`

lseg::insert_iter: No Unfold before Node(val, ptr.next)

lseg::reverse: wrong invariants. even then, does not respond to error message that loop invariant does not hold on entry.

lseg::insert_sorted: changes code logic! node.next = head; return node; to head.next = node; return head;
attempt 4, error depth 3 -- almost got it right. # temperature was 1.5

lseg::insertion_sort: attempt 2, depth 1: almost correct. except unfolding in conditional. (3,2) extra Unfold/Fold.

lseg::split: missing precondition on head being list. Missing Fold(None, None) before Fold(head, None)

lseg::merge: invents Unfold(head1, head2). Does not respond to error message at any temperature.

lseg::merge_sort same problems as list::merge_sort. Having examples with extra fold/unfold in the training data might help.

----------------

Some common issues:
- misses some basic Unfold()/Unfolding() statements.
- does not respond to error messages / line number information.

In [17]:
prompt = '''Nagini is a static verifier for Python. Our aim is to given a statically typed Python program, to come up with appropriate preconditions (e.g. Requires(is_list(head)), Requires(Implies(n is not None, predicate(n)))), postcondition (e.g. Ensures(is_list(Result()))), loop invariants (Invariant(<assertion>)), predicate fold/unfolds (e.g. Fold(is_list(head)) / Unfold(is_list(head))) so that the program verifies correctly. Unfolding(e1, e2) evaluates e2 in the context where predicate e1 is temporarily unfolded.

The user will provide Python code and the verification errors. You must add or change the specifications so that the resulting code verifies correctly. Return only the code without any explanation or wrapping.

The is_list predicate is defined recursively as:
@Predicate
def is_list(head: Node) -> bool:
    return (
        head is not None
        and Acc(head.val)
        and Acc(head.next)
        and Implies(head.next is not None, is_list(head.next))
    )

merge_sort depends on the following methods whose specifications are given:
def count(head: Optional[Node]) -> int:
    """Counts the number of nodes in the list."""
    Requires(Implies(head is not None, is_list(head)))
    Ensures(Implies(head is not None, is_list(head)))
...

def split(head: Optional[Node], idx: int) -> Optional[Node]:
    """Splits the list at the given index. Result is the list starting at idx."""
    Requires(Implies(head is not None, is_list(head)))
    Ensures(Implies(head is not None, is_list(head)))
    Ensures(Implies(Result() is not None, is_list(Result())))
...

def merge(head1: Optional[Node], head2: Optional[Node]) -> Optional[Node]:
    """Merges two sorted lists."""
    Requires(Implies(head1 is not None, is_list(head1)))
    Requires(Implies(head2 is not None, is_list(head2)))
    Ensures(Implies(Result() is not None, is_list(Result())))
...

### Unverified program:
def merge_sort(head: Optional[Node]) -> Optional[Node]:
    """Sorts the list using merge sort."""
    Requires(Implies(head is not None, is_list(head)))
    Ensures(Implies(Result() is not None, is_list(Result())))
    if head is None:
        return None
    if head.next is None:
        return head
    mid = count(head) // 2
    Unfold(is_list(head))
    rest = split(head, mid)
    head = merge_sort(head)
    Unfold(is_list(head))
    rest = merge_sort(rest)
    head = merge(head, rest)
    return head

### Verification error:
Verification failed: Conditional statement might fail. There might be insufficient permission to access head.next. at line 7.7

### Verified program:
'''

In [19]:
print(ev.model.get_response(prompt, temperature=0.1))

def merge_sort(head: Optional[Node]) -> Optional[Node]:
    """Sorts the list using merge sort."""
    Requires(Implies(head is not None, is_list(head)))
    Ensures(Implies(Result() is not None, is_list(Result())))
    if head is None:
        return None
    if Unfolding(is_list(head), head.next is None):
        return head
    mid = count(head) // 2
    Unfold(is_list(head))
    rest = split(head, mid)
    Unfold(is_list(rest))
    head = merge_sort(head)
    Unfold(is_list(head))
    rest = merge_sort(rest)
    head = merge(head, rest)
    return head




In [3]:
ev.run_example("subtree", k=3, n=3, key="tree", examples=[])

Running example: subtree ; attempt: 1 ; error depth: 1
Using temperature: 1.5
Generated program from model:
def subtree(root: Optional[TreeNode], key: int) -> Optional[TreeNode]:
    """Returns the subtree rooted at node with the given key if it exists, None otherwise
    Permissions to the rest of the tree are leaked"""
    Requires(Implies(root is not None, tree(root)))
    Ensures(Implies(Result() is not None, tree(Result())))
    if root is None:
        return None
    Unfold(tree(root))
    if  root.key == key:
        Fold(tree(root))
        return root
    if key < root.key:
        res = subtree(root.left, key)
        Fold(tree(root))
        return res
    res = subtree(root.right, key)
    Fold(tree(root))
    return res

response ['', 'Verification failed', 'Errors:', 'Fold might fail. There might be insufficient permission to access tree(n.left). (tmp.py@48.8)', 'Verification took 2.94 seconds.']
Verification result:
 Verification failed: Fold might fail. There might be 

(True,
 (2, 1),
 'def subtree(root: Optional[TreeNode], key: int) -> Optional[TreeNode]:\n    """Returns the subtree rooted at node with the given key if it exists, None otherwise\n    Permissions to the rest of the tree are leaked"""\n    Requires(Implies(root is not None, tree(root)))\n    Ensures(Implies(Result() is not None, tree(Result())))\n    if root is None:\n        return None\n    Unfold(tree(root))\n    if root.key == key:\n        Fold(tree(root))\n        return root\n    if key < root.key:\n        res = subtree(root.left, key)\n        return res\n    res = subtree(root.right, key)\n    return res\n')